# Bing 7th description

I haven't run the code yet.

In [ ]:
# Importing the necessary modules
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

# Setting the random seed for reproducibility
torch.manual_seed(0)
np.random.seed(0)

# Checking if GPU is available and setting the device accordingly
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Defining the constants for the equation of state and the intervals for sampling
Gamma = 1.4 # Adiabatic index
rho_min = 0 # Lower bound for density
rho_max = 10.1 # Upper bound for density
eps_min = 0 # Lower bound for specific internal energy
eps_max = 2.02 # Upper bound for specific internal energy
vx_min = 0 # Lower bound for velocity in x direction
vx_max = 0.721 # Upper bound for velocity in x direction

# Defining the function to calculate the pressure from the equation of state
def p(rho, eps):
    return (Gamma - 1) * rho * eps

# Defining the function to calculate the conservative variables from the primitive variables
def cons(rho, eps, vx):
    W = 1 / np.sqrt(1 - vx**2) # Lorentz factor
    h = 1 + eps + p(rho, eps) / rho # Specific enthalpy
    D = rho * W # Conserved density
    Sx = rho * h * W**2 * vx # Conserved momentum in x direction
    tau = rho * h * W**2 - p(rho, eps) - D # Conserved energy density
    return D, Sx, tau

# Defining the function to generate the training and test datasets by sampling the primitive variables and calculating the corresponding conservative variables and pressure
def generate_data(n_train, n_test):
    # Sampling the primitive variables uniformly over the intervals
    rho_train = np.random.uniform(rho_min, rho_max, n_train)
    eps_train = np.random.uniform(eps_min, eps_max, n_train)
    vx_train = np.random.uniform(vx_min, vx_max, n_train)
    rho_test = np.random.uniform(rho_min, rho_max, n_test)
    eps_test = np.random.uniform(eps_min, eps_max, n_test)
    vx_test = np.random.uniform(vx_min, vx_max, n_test)

    # Calculating the conservative variables and pressure for each sample
    D_train, Sx_train, tau_train = cons(rho_train, eps_train, vx_train)
    D_test, Sx_test, tau_test = cons(rho_test, eps_test, vx_test)
    p_train = p(rho_train, eps_train)
    p_test = p(rho_test, eps_test)

    # Stacking the conservative variables as inputs and pressure as output
    X_train = np.stack((D_train, Sx_train, tau_train), axis=1)
    y_train = p_train.reshape(-1, 1)
    X_test = np.stack((D_test, Sx_test, tau_test), axis=1)
    y_test = p_test.reshape(-1, 1)

    # Converting the numpy arrays to torch tensors and moving them to device
    X_train = torch.from_numpy(X_train).float().to(device)
    y_train = torch.from_numpy(y_train).float().to(device)
    X_test = torch.from_numpy(X_test).float().to(device)
    y_test = torch.from_numpy(y_test).float().to(device)

    return X_train, y_train, X_test, y_test

# Defining the neural network class with two hidden layers and sigmoid activation functions
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(3, 600) # First hidden layer with 600 neurons
        self.fc2 = nn.Linear(600, 200) # Second hidden layer with 200 neurons
        self.fc3 = nn.Linear(200, 1) # Output layer with 1 neuron
        self.sigmoid = nn.Sigmoid() # Sigmoid activation function
        self.relu = nn.ReLU() # ReLU nonlinearity for the output

    def forward(self, x):
        x = self.sigmoid(self.fc1(x)) # Applying the first hidden layer and sigmoid
        x = self.sigmoid(self.fc2(x)) # Applying the second hidden layer and sigmoid
        x = self.relu(self.fc3(x)) # Applying the output layer and ReLU
        return x

# Creating an instance of the neural network and moving it to device
net = Net().to(device)

# Defining the loss function as mean squared error
criterion = nn.MSELoss()

# Defining the optimizer as Adam with initial learning rate
optimizer = optim.Adam(net.parameters(), lr=6e-4)

# Defining the scheduler to reduce the learning rate on plateau
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, threshold=0.0005, min_lr=1e-6)

# Defining the number of epochs, batch size, and data size
epochs = 400
batch_size = 32
n_train = 80000
n_test = 10000

# Generating the training and test datasets
X_train, y_train, X_test, y_test = generate_data(n_train, n_test)

# Creating lists to store the losses and learning rate for each epoch
train_losses = []
test_losses = []
lrs = []

# Looping over the epochs
for epoch in range(epochs):
    # Shuffling the training data and splitting it into batches
    perm = torch.randperm(n_train)
    X_train = X_train[perm]
    y_train = y_train[perm]
    batches = torch.split(X_train, batch_size)
    targets = torch.split(y_train, batch_size)

    # Initializing the running loss for the training data
    running_loss = 0.0

    # Looping over the batches
    for i, (batch, target) in enumerate(zip(batches, targets)):
        # Zeroing the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        output = net(batch)

        # Calculating the loss
        loss = criterion(output, target)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Updating the running loss
        running_loss += loss.item()

    # Calculating the average loss for the training data
    train_loss = running_loss / len(batches)
    train_losses.append(train_loss)

    # Evaluating the network on the test data
    with torch.no_grad():
        output_test = net(X_test)
        test_loss = criterion(output_test, y_test).item()
        test_losses.append(test_loss)

    # Updating the scheduler based on the test loss
    scheduler.step(test_loss)

    # Getting the current learning rate
    lr = optimizer.param_groups[0]['lr']
    lrs.append(lr)

    # Printing the progress
    print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, LR: {lr:.6f}')

# Defining the function to calculate the L1 and L_inf norms of the error on a data series
def error_norms(y_true, y_pred):
    error = y_true - y_pred # Element-wise error
    L1_norm = torch.mean(torch.abs(error)) # Mean absolute error
    L_inf_norm = torch.max(torch.abs(error)) # Maximum absolute error
    return L1_norm.item(), L_inf_norm.item()

# Calculating the error norms for the training and test data
train_L1_norm, train_L_inf_norm = error_norms(y_train, net(X_train))
test_L1_norm, test_L_inf_norm = error_norms(y_test, net(X_test))

# Printing the error norms
print(f'Train L1 norm: {train_L1_norm:.4f}, Train L_inf norm: {train_L_inf_norm:.4f}')
print(f'Test L1 norm: {test_L1_norm:.4f}, Test L_inf norm: {test_L_inf_norm:.4f}')

# Defining the function to calculate the other primitive variables from pressure and conservative variables using equations A2-A5 from Dieseldorst et al.
def prim(p, D, Sx, tau):
    vx = Sx / (tau + D + p) # Primitive velocity in x direction
    W = 1 / torch.sqrt(1 - vx**2) # Lorentz factor
    eps = (tau + D * (1 - W) + p * (1 - W**2)) / (D * W) # Specific internal energy
    rho = D / W # Density
    return rho, eps, vx

# Calculating the other primitive variables for the training and test data
rho_train_pred, eps_train_pred, vx_train_pred = prim(net(X_train), X_train[:,0], X_train[:,1], X_train[:,2])
rho_test_pred, eps_test_pred, vx_test_pred = prim(net(X_test), X_test[:,0], X_test[:,1], X_test[:,2])

# Calculating the true primitive variables for the training and test data
rho_train_true = X_train[:,0] / torch.sqrt(1 - (X_train[:,1] / (X_train[:,2] + X_train[:,0] + y_train))**2)
eps_train_true = y_train / ((Gamma - 1) * rho_train_true)
vx_train_true = X_train[:,1] / (X_train[:,2] + X_train[:,0] + y_train)
rho_test_true = X_test[:,0] / torch.sqrt(1 - (X_test[:,1] / (X_test[:,2] + X_test[:,0] + y_test))**2)
eps_test_true = y_test / ((Gamma - 1) * rho_test_true)
vx_test_true = X_test[:,1] / (X_test[:,2] + X_test[:,0] + y_test)

# Plotting the mean squared error against epochs for the training and test data
plt.figure(figsize=(10, 6))
plt.plot(range(epochs), train_losses, label='Train')
plt.plot(range(epochs), test_losses, label='Test')
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.title('Mean Squared Error vs Epochs')
plt.legend()
plt.grid()
plt.show()

# Plotting the learning rate against epochs
plt.figure(figsize=(10, 6))
plt.plot(range(epochs), lrs)
plt.xlabel('Epoch')
plt.ylabel('Learning Rate')
plt.title('Learning Rate vs Epochs')
plt.grid()
plt.show()

# Plotting the pressure against density for the training and test data
plt.figure(figsize=(10, 6))
plt.scatter(rho_train_true.cpu(), y_train.cpu(), s=1, label='Train True')
plt.scatter(rho_train_pred.cpu(), net(X_train).cpu(), s=1, label='Train Pred')
plt.scatter(rho_test_true.cpu(), y_test.cpu(), s=1, label='Test True')
plt.scatter(rho_test_pred.cpu(), net(X_test).cpu(), s=1, label='Test Pred')
plt.xlabel('Density')
plt.ylabel('Pressure')
plt.title('Pressure vs Density')
plt.legend()
plt.grid()
plt.show()

# Saving the results and plots
torch.save(net.state_dict(), 'net.pth') # Saving the network parameters
np.savez('results.npz', train_losses=train_losses, test_losses=test_losses, lrs=lrs,
         train_L1_norm=train_L1_norm, train_L_inf_norm=train_L_inf_norm,
         test_L1_norm=test_L1_norm, test_L_inf_norm=test_L_inf_norm,
         rho_train_true=rho_train_true.cpu().numpy(), eps_train_true=eps_train_true.cpu().numpy(), vx_train_true=vx_train_true.cpu().numpy(),
         rho_train_pred=rho_train_pred.cpu().numpy(), eps_train_pred=eps_train_pred.cpu().numpy(), vx_train_pred=vx_train_pred.cpu().numpy(),
         rho_test_true=rho_test_true.cpu().numpy(), eps_test_true=eps_test_true.cpu().numpy(), vx_test_true=vx_test_true.cpu().numpy(),
         rho_test_pred=rho_test_pred.cpu().numpy(), eps_test_pred=eps_test_pred.cpu().numpy(), vx_test_pred=vx_test_pred.cpu().numpy())
plt.savefig('mse.png') # Saving the MSE plot
plt.savefig('lr.png') # Saving the learning rate plot
plt.savefig('p_rho.png') # Saving the pressure-density plot

# Printing a message to indicate that the code is done
print('Code completed.')
